# Re-analysing Yim, Shao & Xu (2024) and using machine learning to learn more about glitch distributions

This notebook is for the analysis of ATNF pulsar and JBCA and ATNF glitch data. We will re-do the analysis of [Yim, Shao & Xu (2024)](https://academic.oup.com/mnras/article/532/4/3893/7712489) but using improved code, including:

- Webscrapping for data so we always have the latest updates included
- Handling the data using Pandas which makes the code more readable and execute faster
- Explore the pulsar dataset more to find what pulsar features are correlated
- Write code to determine the glitch size distribution and waiting time distribution
- Use machine learning to determine the which features can be used to predict the above distributions

The project will be divided into two main parts:

*PART I* (reproducing [Yim, Shao & Xu (2024)](https://academic.oup.com/mnras/article/532/4/3893/7712489))
- Loading the data (using the self-written Python module, <code>read_catalogues.py<code>)
- Cleaning the data
- Exploring/processing/applying mathematical models to the data
- Visualising the results

*PART II* (applying machine learning to determine glitch size and waiting time distributions)
- Determining the each pulsar's actual distribution for glitch sizes and waiting times (creating labels for training and testing)
- Processing the data so it is in a suitable format for applying machine learning models
- Applying different machine learning models
- Evaluating different machine learning models

---

# PART I - Re-analysing Yim, Shao & Xu (2024)

## Pre-amble

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import read_catalogues # Self-written

---

## Loading the data

Loading glitch data from the JBCA Glitch Catalogue:

In [8]:
df_glitch_JBCA = read_catalogues.read_JBCA_glitch_catalogue()
df_glitch_JBCA

,Pulsar name,J-name,No.,MJD,+/-,dF/F,+/-,dF1/F1,+/-,References
0,J0007+7303,0007+7303,1,54953,X,554,1,1.0,0.1,"Abdo+2012 [awd+12], also in Ray+2011 [rkp+11]"
1,J0007+7303,0007+7303,2,55466,X,1260,X,X,X,Belfore+2011 [3rd Fermi symp.]
2,J0040-7335,0040-7335,1,59919.7,X,1.31,0.18,0.056,0.025,New. Also in Carli+24 [cab+24]
3,J0040-7335,0040-7335,2,60355.8,X,1.9,0.4,0.68,0.11,New
4,J0040-7337,0040-7337,1,60013.13,0.05,1810,X,7,X,Carli+24 [cab+24]
...,...,...,...,...,...,...,...,...,...,...
707,1E_2259+586,2301+5852,5,54880,X,-14,1,-29.3,22.2,Icdem+2012 [ibi12]
708,B2323+63,2325+6316,1,53957,31,0.21,0.02,-0.32,0.04,Basu+2021 [bsa+21]
709,B2334+61,2337+6151,1,53642,13,20470,1,23.8,0.4,"Espinoza+2011 [elsk11], also in Yuan+2010 [ymw..."
710,J2346-0609,2346-0609,1,57495,2,0.55,0.01,2.4,0.4,Basu+2021 [bsa+21]


In [9]:
df_glitch_JBCA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Pulsar name  712 non-null    object
 1   J-name       712 non-null    object
 2   No.          712 non-null    object
 3   MJD          712 non-null    object
 4   +/-          712 non-null    object
 5   dF/F         712 non-null    object
 6   +/-          712 non-null    object
 7   dF1/F1       712 non-null    object
 8   +/-          712 non-null    object
 9   References   712 non-null    object
dtypes: object(10)
memory usage: 55.8+ KB


The JBCA Glitch Catalogue has 712 glitch entries in total across 10 different columns (2025/11/09). We will need to convert certain columns to the correct data type, i.e. changing object (string) to float. Although there are apparently no non-null entries, we see above that any null entries are marked by a 'X'. We will make sure to change these into actual null entries shortly.

Download the ATNF files if they are not in the current working directory:

In [12]:
if not os.path.exists('psrcat_pkg.tar.gz'):
    read_catalogues.download_ATNF_catalogues()

Loading glitch data from the ATNF Glitch Catalogue:

In [14]:
df_glitch_ATNF = read_catalogues.read_ATNF_glitch_catalogue()
df_glitch_ATNF

,Name,J2000 Name,Glitch Epoch,+/-,dF_F,+/-,dF1_F1,+/-,Q,+/-,T_d,+/-,Ref.
0,J0007+7303,J0007+7303,54952.652,-,553.7,0.6,0.97,0.06,-,-,-,-,awd+12
1,B0144+59,J0147+5922,53682,15,0.056,0.003,-0.21,0.05,-,-,-,-,ywml10
2,B0154+61,J0157+6212,58283,3,2.6,0.3,-,-,-,-,-,-,bsa+22
3,J0146+6145,J0146+6145,51141,248,650,150,14,5,-,-,-,-,mks05
4,J0146+6145,J0146+6145,53809.185840,-,1630,350,5100,1100,1.1,0.3,17.0,1.1,gdk11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,J2301+5852,J2301+5852,56125,2,260,50,-2600,200,-,-,-,-,akn+13
640,B2323+63,J2325+6316,53957,31,0.21,0.02,-0.32,0.04,-,-,-,-,bsa+22
641,B2334+61,J2337+6151,53615,6,20579.4,1.2,156,4,0.0046,0.0007,21.4,0.5,ymw+10
642,-,-,-,-,-,-,-,-,0.0029,0.0001,147,2,ymw+10


In [15]:
df_glitch_ATNF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 644 entries, 0 to 643
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          644 non-null    object
 1   J2000 Name    644 non-null    object
 2   Glitch Epoch  644 non-null    object
 3   +/-           644 non-null    object
 4   dF_F          644 non-null    object
 5   +/-           644 non-null    object
 6   dF1_F1        644 non-null    object
 7   +/-           644 non-null    object
 8   Q             644 non-null    object
 9   +/-           644 non-null    object
 10  T_d           644 non-null    object
 11  +/-           644 non-null    object
 12  Ref.          644 non-null    object
dtypes: object(13)
memory usage: 65.5+ KB


The JBCA Glitch Catalogue has 644 glitch entries in total across 12 different columns (2025/11/09). Some of these entries are for the same glitch but for several different recovery parameters, e.g. if a glitch had two recovery timescales. For these multi-exponential recovery glitches, we will treat each recovery independently.

Like before, we will need to convert certain columns to the correct data type, i.e. changing object (string) to float. Although there are apparently no non-null entries, we see above that any null entries are marked by a '-'. We will make sure to change these into actual null entries shortly.

Loading pulsar data from the ATNF Pulsar Catalogue:

In [18]:
df_pulsars = read_catalogues.read_ATNF_pulsar_catalogue()
df_pulsars

,A1,A12DOT,A1DOT,A1_2,A1_3,ASSOC,BINARY,BINCOMP,CLK,DECJ,...,T0,T0_2,T0_3,TASC,TASC_2,TAU_SC,TYPE,UNITS,W10,W50
0,None,None,None,None,None,"GRS:4FGL_J0002.8+6217[aab+22],XRS:1XSPS_J00025...",None,None,None,+62:16:09.4,...,None,None,None,None,None,None,HE[wcp+18],None,None,None
1,None,None,None,None,None,None,None,None,None,+18:34:59,...,None,None,None,None,None,None,None,None,112.1,61.3
2,None,None,None,None,None,"GRS:4FGL_J0007.0+7303[aab+22],XRS:RX_J0007.0+7...",None,None,None,+73:03:07.4,...,None,None,None,None,None,None,NRAD[aab+22],None,None,None
3,None,None,None,None,None,None,None,None,None,+08:10,...,None,None,None,None,None,None,None,None,53,13
4,None,None,None,None,None,None,None,None,TT(BIPM2019),+54:31:40,...,None,None,None,None,None,None,RRAT[dcm+23],None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4346,None,None,None,None,None,None,None,None,None,-22:51:53,...,None,None,None,None,None,None,None,None,21,9
4347,8.8929760,None,None,None,None,None,ELL1,He[mzl+23],None,+00:51:09.57,...,59258.1366884,None,None,None,None,None,None,TDB,1.7,0.5
4348,None,None,None,None,None,None,None,None,None,04:43,...,None,None,None,None,None,None,None,None,None,None
4349,None,None,None,None,None,None,None,None,TT(BIPM2019),+15:23:19,...,None,None,None,None,None,None,RRAT[dcm+23],None,None,None


In [19]:
df_pulsars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4351 entries, 0 to 4350
Columns: 162 entries, A1 to W50
dtypes: object(162)
memory usage: 5.4+ MB


We see that there are 4351 entries (pulsars) each with up to 162 features (2025/11/09). Shortly, we will pull out only the features that are important to us. We will convert some columns into the correct data type too.

---

## Cleaning the data

### JBCA Glitch Catalogue

In [60]:
# Changing all X's into NaN
df_glitch_JBCA = df_glitch_JBCA.replace(['X', 'x'], pd.NA)

# Renaming each header
headers = ['pulsar_name', 'J_name', 'pulsar_glitch_number', 'MJD', 'MJD_err', 'dF_F', 'dF_F_err', 'dF1_F1', 'dF1_F1_err', 'references']
df_glitch_JBCA.columns = headers

# Changing each column to its correct data type - use Pandas dtypes as they support NaN values (native Python/NumPy does not)
dtype_map = {
    'pulsar_name' : 'string',
    'J_name' : 'string', 
    'pulsar_glitch_number' : 'Int64', 
    'MJD' : 'Float64', 
    'MJD_err' : 'Float64', 
    'dF_F' : 'Float64', 
    'dF_F_err' : 'Float64', 
    'dF1_F1' : 'Float64', 
    'dF1_F1_err' : 'Float64', 
    'references' : 'string'
}
df_glitch_JBCA = df_glitch_JBCA.astype(dtype_map)

df_glitch_JBCA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   pulsar_name           709 non-null    string 
 1   J_name                709 non-null    string 
 2   pulsar_glitch_number  709 non-null    Int64  
 3   MJD                   709 non-null    Float64
 4   MJD_err               663 non-null    Float64
 5   dF_F                  709 non-null    Float64
 6   dF_F_err              691 non-null    Float64
 7   dF1_F1                594 non-null    Float64
 8   dF1_F1_err            589 non-null    Float64
 9   references            709 non-null    string 
dtypes: Float64(6), Int64(1), string(3)
memory usage: 60.4 KB


We see that there are some issues, for example, there is a missing J-name. We will correct that. There are also two glitches that do not have a dF_F reading, we will get rid of those. 

#### Finding the null J-name and correcting it

In [27]:
df_glitch_JBCA[df_glitch_JBCA['J_name'].isnull()]

,pulsar_name,J_name,pulsar_glitch_number,MJD,MJD_err,dF_F,dF_F_err,dF1_F1,dF1_F1_err,references
562,AX_1838.0-0655,<NA>,1,55010.0,4.0,1550.0,70.0,<NA>,<NA>,Kuiper+2010 [kh10]


In [28]:
df_glitch_JBCA.loc[df_glitch_JBCA['J_name'].isnull(), 'J_name'] = '1838-0655'
df_glitch_JBCA.loc[df_glitch_JBCA['pulsar_name'] == 'AX_1838.0-0655', 'pulsar_name'] = 'J1838-0655'

In [29]:
df_glitch_JBCA.loc[df_glitch_JBCA['J_name'] == '1838-0655', :]

,pulsar_name,J_name,pulsar_glitch_number,MJD,MJD_err,dF_F,dF_F_err,dF1_F1,dF1_F1_err,references
562,J1838-0655,1838-0655,1,55010.0,4.0,1550.0,70.0,<NA>,<NA>,Kuiper+2010 [kh10]


#### Finding the null dF/F values and removing them

In [31]:
df_glitch_JBCA[df_glitch_JBCA['dF_F'].isnull()]

,pulsar_name,J_name,pulsar_glitch_number,MJD,MJD_err,dF_F,dF_F_err,dF1_F1,dF1_F1_err,references
307,B1338-62,1341-6220,34,58178.0,15.0,<NA>,<NA>,<NA>,<NA>,Lower+2021 [ljd+21]
308,B1338-62,1341-6220,35,58214.0,17.0,<NA>,<NA>,<NA>,<NA>,Lower+2021 [ljd+21]


In [32]:
df_glitch_JBCA = df_glitch_JBCA.dropna(subset=['dF_F']).reset_index(drop=True) # Most Pythonic way to drop NaNs
df_glitch_JBCA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 710 entries, 0 to 709
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   pulsar_name           710 non-null    string 
 1   J_name                710 non-null    string 
 2   pulsar_glitch_number  710 non-null    Int64  
 3   MJD                   710 non-null    Float64
 4   MJD_err               663 non-null    Float64
 5   dF_F                  710 non-null    Float64
 6   dF_F_err              692 non-null    Float64
 7   dF1_F1                595 non-null    Float64
 8   dF1_F1_err            590 non-null    Float64
 9   references            710 non-null    string 
dtypes: Float64(6), Int64(1), string(3)
memory usage: 60.5 KB


#### Checking if all dF/F is positive, remove any rows that are not 

In [34]:
df_glitch_JBCA[df_glitch_JBCA['dF_F'] < 0]

,pulsar_name,J_name,pulsar_glitch_number,MJD,MJD_err,dF_F,dF_F_err,dF1_F1,dF1_F1_err,references
705,1E_2259+586,2301+5852,5,54880.0,<NA>,-14.0,1.0,-29.3,22.2,Icdem+2012 [ibi12]


In [35]:
df_glitch_JBCA = df_glitch_JBCA[df_glitch_JBCA['dF_F'] >= 0].reset_index(drop=True)
df_glitch_JBCA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   pulsar_name           709 non-null    string 
 1   J_name                709 non-null    string 
 2   pulsar_glitch_number  709 non-null    Int64  
 3   MJD                   709 non-null    Float64
 4   MJD_err               663 non-null    Float64
 5   dF_F                  709 non-null    Float64
 6   dF_F_err              691 non-null    Float64
 7   dF1_F1                594 non-null    Float64
 8   dF1_F1_err            589 non-null    Float64
 9   references            709 non-null    string 
dtypes: Float64(6), Int64(1), string(3)
memory usage: 60.4 KB


#### Renaming the J-name of J1844+00 to J1844+0034 

In [37]:
df_glitch_JBCA.loc[df_glitch_JBCA['J_name'] == '1844+00']

,pulsar_name,J_name,pulsar_glitch_number,MJD,MJD_err,dF_F,dF_F_err,dF1_F1,dF1_F1_err,references
584,J1844+00,1844+00,1,51435.0,3.0,0.3,0.1,1.0,2.0,Espinoza+2011 [elsk11]
585,J1844+00,1844+00,2,51722.5,0.4,5.2,0.1,4.0,3.0,Espinoza+2011 [elsk11]
586,J1844+00,1844+00,3,56784.0,43.0,249.55,0.03,4.1,0.7,Basu+2021 [bsa+21]


In [62]:
df_glitch_JBCA.loc[df_glitch_JBCA['J_name'] == '1844+00', 'J_name'] = '1844+0034'

In [39]:
df_glitch_JBCA.loc[df_glitch_JBCA['J_name'] == '1844+0034', :]

,pulsar_name,J_name,pulsar_glitch_number,MJD,MJD_err,dF_F,dF_F_err,dF1_F1,dF1_F1_err,references
584,J1844+00,1844+0034,1,51435.0,3.0,0.3,0.1,1.0,2.0,Espinoza+2011 [elsk11]
585,J1844+00,1844+0034,2,51722.5,0.4,5.2,0.1,4.0,3.0,Espinoza+2011 [elsk11]
586,J1844+00,1844+0034,3,56784.0,43.0,249.55,0.03,4.1,0.7,Basu+2021 [bsa+21]


#### Adding 'J' to the start of all J-names

In [58]:
does_not_have_J = ~df_glitch_JBCA['J_name'].str.startswith('J') # Creates a Boolean mask that tests whether J_name starts with 'J', the tilde is a NOT operator so exchanges True <--> False  
df_glitch_JBCA.loc[does_not_have_J, 'J_name'] = 'J' + df_glitch_JBCA.loc[does_not_have_J, 'J_name']

df_glitch_JBCA.head()

,pulsar_name,J_name,pulsar_glitch_number,MJD,MJD_err,dF_F,dF_F_err,dF1_F1,dF1_F1_err,references
0,J0007+7303,J0007+7303,1,54953.0,<NA>,554.0,1.0,1.0,0.1,"Abdo+2012 [awd+12], also in Ray+2011 [rkp+11]"
1,J0007+7303,J0007+7303,2,55466.0,<NA>,1260.0,<NA>,<NA>,<NA>,Belfore+2011 [3rd Fermi symp.]
2,J0040-7335,J0040-7335,1,59919.7,<NA>,1.31,0.18,0.056,0.025,New. Also in Carli+24 [cab+24]
3,J0040-7335,J0040-7335,2,60355.8,<NA>,1.9,0.4,0.68,0.11,New
4,J0040-7337,J0040-7337,1,60013.13,0.05,1810.0,<NA>,7.0,<NA>,Carli+24 [cab+24]


### ATNF Glitch Catalogue

In [ ]:
# Changing all X's into NaN
df_glitch_ATNF.replace('-', np.nan, inplace=True)

In [ ]:
# Changing the column names
headers = ['pulsar_name', 'J_name', 'MJD', 'MJD_err', 'dF_F', 'dF_F_err', 'dF1_F1', 'dF1_F1_err', 'Q', 'Q_err', 'T_d', 'T_d_err', 'references']
df_glitch_ATNF.columns = headers
df_glitch_ATNF.info()

In terms of missing data, it seems okay, but it is worth checking the errors for Q and T_d, as they each have 2 errors missing. Also, there seems to be only 626 pulsar names but 644 references. Perhaps there are some empty entries under references. In fact, the reason for this is because some entries represent multiple recoveries of a single glitch, where each recovery component has its own row. For example, look at Vela's glitch on MJD 51559.3190:

In [ ]:
df_glitch_ATNF.iloc[145:160]

#### Ensuring each recovery component has pulsar data 

One can see that there are 4 components for the glitch with different recovery parameters (Q and T_d). To clean this, we will just copy the other data (pulsar_name, J_name, ..., dF1_F1_err) from the first component, as each component has the same properties. 

In [ ]:
no_name_indices = df_glitch_ATNF[df_glitch_ATNF['pulsar_name'].isnull()].index

In [ ]:
for index in no_name_indices:
    test_index = index - 1
    df_glitch_ATNF.loc[index, 'pulsar_name': 'dF1_F1_err'] = df_glitch_ATNF.loc[test_index, 'pulsar_name': 'dF1_F1_err']

In [ ]:
df_glitch_ATNF.iloc[145:160]

#### Checking missing Q_err and T_d_err values

In [ ]:
df_glitch_ATNF[df_glitch_ATNF['Q'].notnull() & df_glitch_ATNF['Q_err'].isnull()]

Having checked the above rows in the raw data file, I can confirm that these rows do not have Q_err. (There is nothing wrong with how the code read in the values.)

In [ ]:
df_glitch_ATNF[df_glitch_ATNF['T_d'].notnull() & df_glitch_ATNF['T_d_err'].isnull()]

Having checked the above rows in the raw data file, I can confirm that these rows do not have T_d_err. (There is nothing wrong with how the code read in the values.)

#### Checking for NaNs and negative dF_F

In [ ]:
df_glitch_ATNF['dF_F'] = pd.to_numeric(df_glitch_ATNF['dF_F'], errors = 'coerce') # Coerce leaves any non-numerical values as NaN
df_glitch_ATNF[df_glitch_ATNF['dF_F'].isnull()]

In [ ]:
indices_to_drop = df_glitch_ATNF[df_glitch_ATNF['dF_F'].isnull()].index
df_glitch_ATNF.drop(indices_to_drop, inplace = True)
df_glitch_ATNF.reset_index(inplace = True, drop = True)

In [ ]:
df_glitch_ATNF[df_glitch_ATNF['dF_F'] <= 0]

In [ ]:
indices_to_drop = df_glitch_ATNF[df_glitch_ATNF['dF_F'] <= 0].index
df_glitch_ATNF.drop(indices_to_drop, inplace = True)
df_glitch_ATNF.reset_index(inplace = True, drop = True)

In [ ]:
df_glitch_ATNF.info()

### ATNF Pulsar Catalogue